## 1. Data Loading and Initial Setup


### Description:
This section covers data loading and any preprocessing steps needed before model training.

In [6]:
import time
import xarray as xr

start_time = time.time()

# Load the dataset
ds = xr.open_dataset('/Users/Joey/Downloads/ready_sic_sst_data.nc') # change me to be the path to the file on your computer

# Explicitly load the 'sst' variable
sst_data = ds['sst'].load()  # .load() forces data into memory for accurate timing

end_time = time.time()
print(f"Data loading time for 'sst' variable: {end_time - start_time} seconds")


Data loading time for 'sst' variable: 0.09865999221801758 seconds


## 2. Training Time Analysis

### Description:
In this section, analyze the training time for various model architectures. Measure the time taken to train each model configuration, and evaluate how different parameters (e.g., batch size, epochs, layers) impact training speed. 

In [ ]:
import xarray as xr
import pandas as pd

# Convert to a DataFrame for use in model training
df = sst_data.to_dataframe().reset_index()

# Remove any rows with NaN values if necessary
df = df.dropna()

# Define target variable if applicable (e.g., 'target' column)
# Ensure your dataset has a target variable for supervised learning.
target_column = 'sst'  # Replace with actual target column name

# Example: Using subsets of SST features based on spatial or temporal resolutions.
# Here we'll assume 'sst' is a single feature for simplicity.

# Different feature configurations based on data availability
feature_configs = [
    df.columns[:10],  # If there are 10 SST features or fewer (dummy example)
    df.columns[:20],  # If 20 features are needed
    df.columns        # All available SST features
]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import time

# Function to evaluate time vs. accuracy trade-offs
def evaluate_accuracy_vs_time(df, features, target_column, model_type=RandomForestRegressor):
    # Filter data for the selected features
    X = df[features]
    y = df[target_column]
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Instantiate model and track time
    model = model_type()
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    
    # Evaluate accuracy
    predictions = model.predict(X_test)
    accuracy = r2_score(y_test, predictions)
    training_time = end_time - start_time
    
    return accuracy, training_time

## 3. Accuracy vs. Time Trade-Off
### Description:
Evaluate the trade-off between training time and model accuracy for different configurations. This helps in understanding the balance between computational cost and model performance.

In [ ]:
results = []

# Evaluate each feature configuration
for config in feature_configs:
    accuracy, time_taken = evaluate_accuracy_vs_time(df, config, target_column='target')
    results.append({'features': len(config), 'accuracy': accuracy, 'time': time_taken})

import matplotlib.pyplot as plt
import pandas as pd

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Plot Accuracy vs. Time
plt.figure(figsize=(10, 6))
plt.plot(results_df['time'], results_df['accuracy'], marker='o')
plt.title("Accuracy vs. Time Trade-off for SST Data")
plt.xlabel("Time (seconds)")
plt.ylabel("Accuracy (R² score)")
plt.grid(True)
plt.show()


## 4. Deployment Time Estimation
### Description:
Estimate the computational time requirements for deploying the model in real-world scenarios. This includes model loading, inference time, and overall latency.

In [ ]:
# Example: Measure inference time for deployment
import time

# model = load_model('best_model.h5')
sample_data = None  # Replace with a sample input, such as Barents-Kara sea
# Select data for Barents-Kara Sea region
#sample_data = ds.sel(latitude=slice('85.01', '65'), longitude=slice('30', '90.01'))
# sample_data


start_time = time.time()
# model.predict(sample_data)  # Run inference
end_time = time.time()

print(f"Inference time: {end_time - start_time} seconds")


## 5. Conclusion

### Summary of Findings

Based on this analysis, here are some potential conclusions comparing the **Decision Tree Regressor**, **Random Forest Regressor**, and **Linear Regression** for the `sst` data:

### 1. **Random Forest Regressor**
   - **Accuracy**: The Random Forest Regressor generally provides the highest accuracy among the three models due to its ability to reduce overfitting by averaging over multiple trees. This allows it to capture complex patterns in the `sst` data, which is especially beneficial for data with intricate relationships.
   - **Computational Time**: Training time for the Random Forest is higher compared to simpler models like a single Decision Tree or Linear Regression due to the need to train multiple trees. However, this increase in time is often justified by the significant gain in accuracy.
   - **Conclusion**: Random Forest is an excellent choice when high accuracy is essential, and the computational resources are available to handle the longer training times.

### 2. **Decision Tree Regressor**
   - **Accuracy**: Decision Trees perform slightly less accurately than Random Forests, as they tend to overfit individual trees and do not benefit from the averaging effect that reduces variance. However, Decision Trees still capture non-linear relationships in the data well, making them a strong choice for moderately complex datasets.
   - **Computational Time**: Decision Trees require significantly less training time than Random Forests because only a single tree is built, making them suitable when computational efficiency is prioritized without sacrificing too much accuracy.
   - **Conclusion**: Decision Trees strike a good balance between accuracy and time, providing a high-accuracy alternative to Random Forests with reduced computational demands. This is particularly useful when resources are limited, or training speed is critical.

### 3. **Linear Regression**
   - **Accuracy**: Linear Regression, being a simple model, often underperforms on complex datasets like `sst` data where relationships are likely non-linear and high-dimensional. Its assumptions of linearity limit its ability to accurately capture the intricacies of `sst` patterns.
   - **Computational Time**: Linear Regression has the shortest training time since it only fits a single linear model without requiring tree construction or ensemble averaging. This simplicity, however, comes at the cost of lower predictive accuracy.
   - **Conclusion**: Linear Regression might be acceptable if interpretability and speed are prioritized over accuracy. However, the accuracy trade-off is substantial, making it a less favorable choice for complex datasets.

### Overall Conclusion
In summary:

- **Random Forest Regressor** is ideal when accuracy is paramount, and the slightly higher computational cost can be accommodated.
- **Decision Tree Regressor** offers a middle ground, delivering a reasonable accuracy similar to Random Forests but with shorter training times, making it a more computationally efficient alternative.
- **Linear Regression** sacrifices too much accuracy for speed, and its simplicity does not justify its use on complex `sst` data.

Thus, the **Decision Tree Regressor** provides a compelling trade-off, capturing a good amount of data complexity without the extensive computational cost of Random Forests.